In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name='udacity-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

print(ws)

Workspace name: quick-starts-ws-158927
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-158927
Workspace.create(name='quick-starts-ws-158927', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-158927')


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "my-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

In [4]:
from azureml.core import Environment, ScriptRunConfig

env = Environment.get(workspace = ws, name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [3]:
# SKLearn estimator is deprecated. Use ScriptRunConfig instead.
"""
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
              compute_target=compute_target,
              vm_size="Standard_D2_V2",
              entry_script="train.py")
"""

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
src = ScriptRunConfig(source_directory = ".",
                      script = "train.py",
                      compute_target = "my-compute-cluster",
                      environment = env
                     )

In [7]:
# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
                             "--max_iter":choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=1)

In [8]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             policy=policy,
                             max_total_runs=16,
                             max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_run = exp.submit(hd_config)

# The widget RunDetails can be used to monitor the run
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931
Web View: https://ml.azure.com/runs/HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158927/workspaces/quick-starts-ws-158927&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931
Web View: https://ml.azure.com/runs/HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158927/workspaces/quick-starts-ws-158927&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931',
 'target': 'my-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-21T08:39:48.699348Z',
 'endTimeUtc': '2021-09-21T08:50:26.94762Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0858e017-22cc-4a68-90db-566d17dca7f2',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_13',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg158927.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931/azureml-logs/hyperdrive.txt?sv=2019-07-07

In [11]:
import joblib
# Get your best run and save the model from that run.

print(hd_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_13', 'hyperparameters': '{"--C": 100, "--max_iter": 100}', 'best_primary_metric': 0.9088012139605463, 'status': 'Completed'}, {'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_14', 'hyperparameters': '{"--C": 1, "--max_iter": 100}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}, {'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_6', 'hyperparameters': '{"--C": 200, "--max_iter": 100}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}, {'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_2', 'hyperparameters': '{"--C": 0.01, "--max_iter": 100}', 'best_primary_metric': 0.9084977238239758, 'status': 'Completed'}, {'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_1', 'hyperparameters': '{"--C": 50, "--max_iter": 300}', 'best_primary_metric': 0.9081942336874052, 'status': 'Completed'}, {'run_id': 'HD_69c2b90c-09c9-47b7-b29a-fcb7fd21c931_12', 'hyperparameters': '{"--C": 0.001, "--max_iter": 100}', 'be

In [12]:
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_56ed71ccf2aaa24eb67833e98c077dcd9e084cd7ea2c1db6f1979498693b5da5_d.txt',
 'azureml-logs/65_job_prep-tvmps_56ed71ccf2aaa24eb67833e98c077dcd9e084cd7ea2c1db6f1979498693b5da5_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_56ed71ccf2aaa24eb67833e98c077dcd9e084cd7ea2c1db6f1979498693b5da5_d.txt',
 'logs/azureml/95_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files([path])

In [14]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=2)

In [16]:
# Submit your automl run

automl_run = exp.submit(automl_config)

RunDetails(automl_run).show()

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c4a9cffb-093a-44dc-b6da-888e604eca98,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [17]:
best_run, fitted_model = automl_run.get_output()

In [18]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c4a9cffb-093a-44dc-b6da-888e604eca98_30,,Completed,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [19]:
# Cluster cleanup
compute_target.delete()